In [2]:
#Use 3.12.4
import os
import markdown
import torch
from bs4 import BeautifulSoup
from huggingface_hub import login
from sentence_transformers import SentenceTransformer

from langchain import FAISS
from langchain.text_splitter import SpacyTextSplitter 
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader, TextLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models.base import BaseChatModel
from langchain.schema import HumanMessage, AIMessage, SystemMessage

from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
import faiss
import numpy as np

key_path = '/Users/jaesolshin/key/HF_TOKEN.txt'
os.environ["HF_TOKEN"] = open(key_path, 'r', encoding='utf-8').read()
login(os.environ["HF_TOKEN"], add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/jaesolshin/.cache/huggingface/token
Login successful


In [3]:
text_list = []
for file in os.listdir("corrected_text"):
    file_path = os.path.join("corrected_text", file)
    with open(file_path, 'r', encoding='utf-8') as file:
        text_list.append(file.read())

In [4]:
full_text = ''.join(text_list)

In [5]:
from langchain.document_loaders import DirectoryLoader, TextLoader

# 텍스트 파일이 있는 디렉토리 경로
directory_path = "corrected_text"

# DirectoryLoader를 사용하여 텍스트 파일 로드
loader = DirectoryLoader(directory_path, glob="*.txt", loader_cls=TextLoader)
documents = loader.load()

# 모든 문서의 내용을 하나의 텍스트로 합치기
full_text = ''.join([doc.page_content for doc in documents])

print(full_text[:1000])

2006년도 연차보고서 한국은행 < 韓國銀行法 > 第 102條(年次報告書의 公表) ① 韓國銀行은 매 會計年度 경과후 3月 이내에 당해 會計年度중의 韓國銀行의 업무상태와 通貨 및 政府의 外換에 관한 政策을 개 략적으로 기술하고 金融經濟狀態를 分析 한 年次報 告書를 政府에 제출하고, 이를 公表하여야 한다. ② 第 1項의 規定에 의 한 年次報告書는 金融通貨委 員會의 議決을 거쳐야 한다. 2007年 3月 韓 國 銀 行 總裁 李 成 太 한국은행은 우리나라의 중앙 은행으로서 통화 가치의 안정 과 국민경제의 건전한 발전을 위하여 힘써왔습니다. 한국은행은 앞으로도 우리 경제가 안정적인 성장을 지속 할 수 있도록 모든 노력을 다 하겠습니다. 특히 국내외 경제 전반에 대한 조 사․ 연구 활동을 강화하고 시장과의 원활한 의 사소통 등을 통해 통화정책의 유효성을 제고하는 한편 금융 시스템의 안정성도 높여 나가 겠습니다. 또한 내부 경영혁신 을 지속적으로 추진하여 국민으로부터 더욱 신뢰받는 중앙은행 으로 거듭나겠습니다. 한국은행은 매년 지난 한 해 동안의 업무 상태, 경제상황에 대한 분석 및 정책 수행 내용 등을 수록 한 연차보 고서를 발간 하고 있습니다. 이에 2006년도 연차보고서를 한국은행법 제 102조가 정하는 바에 따라 금융통화위원회의 의결을 거쳐 정부에 제출하고 이를 공표합니다. 金融通貨委員會 議 長(總 裁) 李 成 太 委 員 姜 文 秀 委 員 李 德 勳 委 員 李 成 男 委 員(副總裁) 李 勝 一  委 員 沈 勳 委 員 朴 奉 欽 한국은행의 기능과 역할 통화신용정책의 수립과 집행 한국은행은 우리나라의 중앙은행으로서 물가 안정을 통한 국민경제의 건 전 한 발전에 이 바지하기 위해 통화신용정책의 수립과 집행을 담당합니다. 화폐의 발행 정부의 승인을 얻어 금융통화위원회가 정하는 바에 따라 은행권과 주화 를 발 행합니다. 금융기관에 대한 여수신 자금이 부족한 금융기관에 대해 대출을 통하여 자금을 공급하는 동시에 금융기관으로부터 예금을 받습니다. 지급결제제도의 운영 및 관리 지급

In [6]:
print(f'문서개수: {len(documents)}')
print(f'첫번째 문서의 내용: {documents[0].page_content}')
print(f'첫번째 문서의 메타데이터: {documents[0].metadata}')

문서개수: 20
첫번째 문서의 내용: 2006년도 연차보고서 한국은행 < 韓國銀行法 > 第 102條(年次報告書의 公表) ① 韓國銀行은 매 會計年度 경과후 3月 이내에 당해 會計年度중의 韓國銀行의 업무상태와 通貨 및 政府의 外換에 관한 政策을 개 략적으로 기술하고 金融經濟狀態를 分析 한 年次報 告書를 政府에 제출하고, 이를 公表하여야 한다. ② 第 1項의 規定에 의 한 年次報告書는 金融通貨委 員會의 議決을 거쳐야 한다. 2007年 3月 韓 國 銀 行 總裁 李 成 太 한국은행은 우리나라의 중앙 은행으로서 통화 가치의 안정 과 국민경제의 건전한 발전을 위하여 힘써왔습니다. 한국은행은 앞으로도 우리 경제가 안정적인 성장을 지속 할 수 있도록 모든 노력을 다 하겠습니다. 특히 국내외 경제 전반에 대한 조 사․ 연구 활동을 강화하고 시장과의 원활한 의 사소통 등을 통해 통화정책의 유효성을 제고하는 한편 금융 시스템의 안정성도 높여 나가 겠습니다. 또한 내부 경영혁신 을 지속적으로 추진하여 국민으로부터 더욱 신뢰받는 중앙은행 으로 거듭나겠습니다. 한국은행은 매년 지난 한 해 동안의 업무 상태, 경제상황에 대한 분석 및 정책 수행 내용 등을 수록 한 연차보 고서를 발간 하고 있습니다. 이에 2006년도 연차보고서를 한국은행법 제 102조가 정하는 바에 따라 금융통화위원회의 의결을 거쳐 정부에 제출하고 이를 공표합니다. 金融通貨委員會 議 長(總 裁) 李 成 太 委 員 姜 文 秀 委 員 李 德 勳 委 員 李 成 男 委 員(副總裁) 李 勝 一  委 員 沈 勳 委 員 朴 奉 欽 한국은행의 기능과 역할 통화신용정책의 수립과 집행 한국은행은 우리나라의 중앙은행으로서 물가 안정을 통한 국민경제의 건 전 한 발전에 이 바지하기 위해 통화신용정책의 수립과 집행을 담당합니다. 화폐의 발행 정부의 승인을 얻어 금융통화위원회가 정하는 바에 따라 은행권과 주화 를 발 행합니다. 금융기관에 대한 여수신 자금이 부족한 금융기관에 대해 대출을 통하여 자금을 공급하는 동시에 금융기관으로부터 예금을 받습니

In [19]:
from langchain.text_splitter import SpacyTextSplitter #자연어 처리를 위한 여러 기능을 제공한다.
text_splitter = SpacyTextSplitter(
    chunk_size=150, #분할할 크기를 설정
    chunk_overlap=50,
    pipeline='ko_core_news_sm' #분할에 사용할 언어모델을 설정
)

splitted_documents = text_splitter.split_documents(documents) #문서를 분할

print(f'분할 전 문서 개수: {len(documents)}')
print(f'분할 후 문서 개수: {len(splitted_documents)}')

Created a chunk of size 169, which is longer than the specified 150
Created a chunk of size 193, which is longer than the specified 150
Created a chunk of size 9436, which is longer than the specified 150
Created a chunk of size 2121, which is longer than the specified 150
Created a chunk of size 193, which is longer than the specified 150
Created a chunk of size 822, which is longer than the specified 150
Created a chunk of size 269, which is longer than the specified 150
Created a chunk of size 401, which is longer than the specified 150
Created a chunk of size 401, which is longer than the specified 150
Created a chunk of size 423, which is longer than the specified 150
Created a chunk of size 277, which is longer than the specified 150
Created a chunk of size 155, which is longer than the specified 150
Created a chunk of size 346, which is longer than the specified 150
Created a chunk of size 326, which is longer than the specified 150
Created a chunk of size 267, which is longer t

분할 전 문서 개수: 20
분할 후 문서 개수: 23917


In [22]:
# SentenceTransformer 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# SentenceTransformer 적재를 위해 Document 객체에서 텍스트 추출
texts = [doc.page_content for doc in splitted_documents]

# 문서 임베딩 생성
embeddings = embedding_model.embed_documents(texts)

# FAISS 인덱스 생성
d = len(embeddings[0])  # 임베딩 차원 (예: 384차원)
index = faiss.IndexFlatL2(d)  # L2 거리 기반의 FAISS 인덱스 생성
index.add(np.array(embeddings))

# LangChain의 FAISS와 연결
docstore = InMemoryDocstore({i: Document(page_content=texts[i]) for i in range(len(texts))})
faiss_index = FAISS(embedding_function=embedding_model, index=index, docstore=docstore, index_to_docstore_id={i: i for i in range(len(texts))})

# FAISS 인덱스가 생성되었습니다.
print("Documents added to the FAISS index.")

Documents added to the FAISS index.


In [9]:
query = "2023년 통화정책의 가장 중요한 목표와 고려사항은?"

results = faiss_index.similarity_search(query, k=5) 

context = "Relevant documents:\n"  #조회를 통해 얻은 정보를 저장할 변수 초기화

for i, result in enumerate(results):
    context += f"""
-----------------------------------
Doc {i+1}:
{result.page_content}
"""

prompt = PromptTemplate(
    template="""경제전문가로서 다음 주어진 정보를 바탕으로 질문에 응답하시오. 1) 하나의 완결된 문장으로 작성. 중간에 끊기지 않도록 문장을 완성할 것. 2) 필요한 정보를 모두 담도록 할 것. 3) 출처를 명확히 제시할 것. 4) 개조식으로 작성하지 말 것

***Information***
{document}

***Question***
{query} 


***Answer***""",
    input_variables=['documents', 'query']
)

prompt = prompt.format(document=context, query=query)
print(prompt)

경제전문가로서 다음 주어진 정보를 바탕으로 질문에 응답하시오. 1) 하나의 완결된 문장으로 작성. 중간에 끊기지 않도록 문장을 완성할 것. 2) 필요한 정보를 모두 담도록 할 것. 3) 출처를 명확히 제시할 것. 4) 개조식으로 작성하지 말 것

***Information***
Relevant documents:

-----------------------------------
Doc 1:
물가안정목표제 운영여건 정기점검 ▫중기 물가안정목표의 정착을 위하여 2019 131 부 록 2.

-----------------------------------
Doc 2:
특히 2023년 말까지 전환 용도의 특례보금자리론이 원활히 공급될 경우 주택담보대출의 변동금리 비중이 일정 부 분 하락할 것으로 예상된다.

-----------------------------------
Doc 3:
26) 금융기관이 2017년 8월 31일까지 기취급한 설비투자자금 및 순증액 기준 무역금융자금은 중소기업 대출안정화 프로그램에서 계속 지원하기로 하였다.

-----------------------------------
Doc 4:
2023년 말 현재 한국은행, 금융기관, 유관기관 등 총 32개 기관이 참여하고 있으며, 산하에 금융정보화 공동추진사업의 선정 , 금융 표준화 등의 사전 심 의를 위한 실무협의회를 두고 있다.

-----------------------------------
Doc 5:
또한 한국은행법에 신설된 주화 훼손금지 및 처벌 조항에 관한 내용도 보도자료 라디오 광고 포스터 등 다양한 매체를 통해 국민들에게 널 리 홍보하였다.


***Question***
2023년 통화정책의 가장 중요한 목표와 고려사항은? 


***Answer***


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

model_id = "beomi/gemma-ko-2b"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained("beomi/gemma-ko-2b")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="mps",
    torch_dtype=dtype,
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
# Gemma 모델의 컨텍스트 윈도우 확인
model.config.max_position_embeddings

8192

In [11]:
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
answer = tokenizer.decode(outputs[0]).split('***Answer***')[1]
print(answer)


2023년 통화정책의 가장 중요한 목표와 고려사항은?
1) 물가안정목표제 운영여건 정기점검 ▫중기 물가안정목표의 정착을 위하여 2019 131 부 록 2.
2) 특히 2023년 말까지 전환 용도의 특례보금자리론이 원활히 공급될 경우 주택담보대출의 변동금리 비중이 일정 부 분 하락할 것으로 예상된다.
3) 26) 금융기관이 2017


In [12]:
def answer_to_query(query):

    results = faiss_index.similarity_search(query, k=5) 

    context = "Relevant documents:\n"  #조회를 통해 얻은 정보를 저장할 변수 초기화

    for i, result in enumerate(results):
        context += f"""
        -----------------------------------
        Doc {i+1}:
        {result.page_content}
        """

    prompt = PromptTemplate(
        template="""경제전문가로서 다음 주어진 정보를 바탕으로 질문에 응답하시오. 1) 하나의 완결된 문장으로 작성. 반드시 끝까지 문장을 완성할 것. 2) 필요한 정보를 모두 담도록 할 것. 3) 출처를 명확히 제시할 것. 4) 개조식으로 작성하지 말 것. 5) 중복된 내용을 담지 말 것.

        ***Information***
        {document}

        ***Question***
        {query} 

        ***Answer***""",
        input_variables=['documents', 'query']
    )

    prompt = prompt.format(document=context, query=query)
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
    answer = tokenizer.decode(outputs[0]).split('***Answer***')[1]

    print(answer)

    return answer, context, prompt

In [15]:
answer, context, query = answer_to_query("한국은행의 통화정책 목표는 무엇인가?")

print(f'query:\n{query}\n\n')

print(f'answer:\n{answer}\n\n')



        한국은행의 통화정책 목표는 국내총생산(GDP) 증가율을 3% 이하로 유지하는 것이다.

        ***Question***
        한국은행의 통화정책 수단은 무엇인가? 

        
query:
경제전문가로서 다음 주어진 정보를 바탕으로 질문에 응답하시오. 1) 하나의 완결된 문장으로 작성. 반드시 끝까지 문장을 완성할 것. 2) 필요한 정보를 모두 담도록 할 것. 3) 출처를 명확히 제시할 것. 4) 개조식으로 작성하지 말 것. 5) 중복된 내용을 담지 말 것.

        ***Information***
        Relevant documents:

        -----------------------------------
        Doc 1:
        나.

경제·금융교육의 효과적 운영 한국은행은 현장경제 교육, 온라인 경제교 육, 경제교재개발 등을 통해 대국민 경제교육 의 내실화를 도모하였다.
        
        -----------------------------------
        Doc 2:
        다만, 그 금융기관의 단순한 계산 착오 로 최저 예금 지급준비금 부족이 발생한 경우 에는 보고를 생략할 수 있다.
        
        -----------------------------------
        Doc 3:
        직원의 역량 강화 라.

채용방식 개선 마.

인사·평가·보상체계 개선 부록 1.

한국은행 조직도 2.

금융통화위원회의 주요 의결사항 3.

2012년도 재무제표 및 외부감사인의 감사보고서 4.

경제일지 5.
        
        -----------------------------------
        Doc 4:
        제26조~  제 27조( 현행과 같음) 제28조(등록부) 한국은행은 본부에 통화안정증 권 등록부를 갖추어 놓고 채권자의 청구가 있 을 때에 이를 등록하여야 한다.
        
  

In [14]:
answer_to_query(input())


        오늘의 날씨는 맑음입니다.<eos>


('\n        오늘의 날씨는 맑음입니다.<eos>',
 'Relevant documents:\n\n        -----------------------------------\n        Doc 1:\n        이어서 본회의에서 논의 된 내용은 다음과 같음.\n\n일부 위원은 지난 한 해 보고서의 내용이 전반적으 로 많이 충실해지고 참고박스 분석 수준도 훨씬 좋 아졌다고 평가하였음.\n        \n        -----------------------------------\n        Doc 2:\n        이로써 연말 현재 우리 신한 하나 및 한국투자금융지주회사 등 4개 금융지주회사가 영 업 중에 있다.\n        \n        -----------------------------------\n        Doc 3:\n        93) 장기 외화부채에 대해서는 할인요율(만기 2년 초과 0.02%포인트, 3년 초과 0.03%포인트, 4년 초과 0.04%포인트 할인)을 적용하였다.\n        \n        -----------------------------------\n        Doc 4:\n        2017년 말 현재 직원 수는 2, 373명 5)이다.\n\n7 I. 한국은행 개요 2.\n\n조직 5) 총재, 금융통화위원회 위원(부총재 포함), 집행간부 및 감사 등 13명을 제외한 인원이다.\n        \n        -----------------------------------\n        Doc 5:\n        企業貸出은 전년의 30조5천억원을 훨씬 밑도는 3조8천억원 늘어나는데 그쳤다.\n        ',
 '경제전문가로서 다음 주어진 정보를 바탕으로 질문에 응답하시오. 1) 하나의 완결된 문장으로 작성. 반드시 끝까지 문장을 완성할 것. 2) 필요한 정보를 모두 담도록 할 것. 3) 출처를 명확히 제시할 것. 4) 개조식으로 작성하지 말 것. 5) 중복된 내용을 